In [1]:

import torch
import numpy as np
import random
from transformer_lens import HookedTransformer, ActivationCache
import transformer_lens.utils as utils
import plotly.express as px


SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# %%
# model = HookedTransformer.from_pretrained('gemma-7b', dtype=torch.float16)
model = HookedTransformer.from_pretrained('gpt2-small', dtype=torch.float32)

/home/slava/fun/multiple_choice/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [4]:

from atp import Patching
model.eval()
model.set_use_attn_result(True)
model.set_use_attn_in(True)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)

from importlib import reload
import atp
reload(atp)

patching = atp.Patching(model, 'atp')

In [2]:
from datasets import load_dataset

dataset = load_dataset("cais/mmlu", "high_school_biology")

In [3]:
letters = ['A', 'B', 'C', 'D']

prelude = "A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. "
post_text = "Answer: ("

for i in range(100):
  question = dataset["test"][i]["question"] + ':\n'

  answers = f"""
(A) {dataset["test"][i]["choices"][0]}
(B) {dataset["test"][i]["choices"][1]}
(C) {dataset["test"][i]["choices"][2]}
(D) {dataset["test"][i]["choices"][3]}
"""

#   correct_answers.append(dataset["test"][i]["answer"])

#   prompts.append(prelude + question + answers + post_text)

In [6]:
def get_prompt_pair(i):
    correct_id = dataset["test"][i]["answer"]
    correct_answer = dataset["test"][i]["choices"][correct_id]

    clean_answers = f"""
(A) {dataset["test"][i]["choices"][0]}
(B) {dataset["test"][i]["choices"][1]}
(C) {dataset["test"][i]["choices"][2]}
(D) {dataset["test"][i]["choices"][3]}
"""

    corrupted_answers = f"""
(A) {dataset["test"][i]["choices"][1]}
(B) {dataset["test"][i]["choices"][2]}
(C) {dataset["test"][i]["choices"][3]}
(D) {dataset["test"][i]["choices"][0]}
"""

    clean = prelude + question + clean_answers + post_text
    corrupted = prelude + question + corrupted_answers + post_text

    return clean, corrupted, letters[correct_id]




In [7]:
x_clean, x_corr, a_clean = get_prompt_pair(0)
print(x_clean)
print(x_corr)
print(a_clean)

A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. Which of the following theories is based on the notion that mitochondria and chloroplasts evolved from prokaryotic cells?:

(A) directional selection.
(B) stabilizing selection.
(C) sexual selection.
(D) disruptive selection.
Answer: (
A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. Which of the following theories is based on the notion that mitochondria and chloroplasts evolved from prokaryotic cells?:

(A) stabilizing selection.
(B) sexual selection.
(C) disruptive selection.
(D) directional selection.
Answer: (
A


In [10]:
# patch z

patching.patching(x_clean, a_clean, x_corr, "C", component='attn_z')

Clean logit difference: 2.352
Corrupted logit difference: 2.355
Patching...


In [11]:
patching.plot_patch(width=1200, height=1400, layer=10)